# YT
自動下載該頻道所有播放清單的影片，並依播放清單名稱儲存 
output_path = 儲存路徑 
channel_name = 頻道名稱

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import subprocess

output_path = "save_path"
channel_name = "channel_name"

url = 'https://www.youtube.com/'+channel_name+'/playlists'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
scripts = soup.find_all("script")
target = ""
for script in scripts:
    # 檢查是否包含ytInitialData
    if script.string==None:
        continue
    if script.string.find("ytInitialData")!=-1:
        # 使用正規表達式或其他方法獲得ytInitialData的值
        data_start = script.string.find('ytInitialData') + len('ytInitialData = ')
        data_end = script.string.find(';', data_start)
        yt_initial_data_str = script.string[data_start:data_end].strip()
        target = json.loads(yt_initial_data_str)
        break
target = target['contents']['twoColumnBrowseResultsRenderer']['tabs'][5]['tabRenderer']['content']['sectionListRenderer']['contents'][0]['itemSectionRenderer']['contents'][0]['gridRenderer']['items']
playlist = {}
for i in target:
    title = i['gridPlaylistRenderer']['title']['runs'][0]['text']
    url = "https://www.youtube.com/playlist?list="+i['gridPlaylistRenderer']['playlistId']
    playlist[title] = url

command = 'yt-dlp -i -f bestvideo+bestaudio -o ' 
for i in playlist:
    complete_command = command + output_path + i +"\\%(title)s.%(ext)s " + '"' + playlist.get(i) + '"'
    print("正在下載: "+i)
    print(complete_command)
    try:
        subprocess.run(complete_command, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"命令執行失敗: {e}")